# Setup

## Libraries / Settings

In [1]:
import pandas as pd
import numpy as np
import json, requests, urllib, io
from requests.structures import CaseInsensitiveDict
from pathlib import Path
import subprocess
from collections import defaultdict

In [2]:
import seaborn as sns
from matplotlib.patches import Patch
import matplotlib.pyplot as plt

In [3]:
base_dir = Path("..")
graph_path = base_dir / "output" / "original-graphs"
input_dir = base_dir / "input"

In [21]:
# Path to binary of https://github.com/carissableker/thresholding
THRESHOLD_EXE = "./thresholding_analysis"

In [5]:
import design

## File names, locations, content

In [6]:
timepts = "ALL"
graphs_infos = {}
_file_contents = defaultdict(list)

for contrast in design.files:
    _file = design.files[contrast]

    D = {"treatments":{}, "contrasts":{}}
    
    g = f"{contrast}.{timepts}"
    D["contrasts"][g] = {
        "contrast": contrast, 
        "time_pts": timepts, 
        "og_file": design.files[contrast],
        "file": f"{g}.ncol"
    }  
    
    treatments = contrast.split("-v-")
    for t in treatments:
        g = f"{contrast}.{timepts}.{t}"
        D["treatments"][g] = {
            "contrast": contrast, 
            "time_pts": timepts, 
            "treatment": t,
            "og_file": design.files[contrast],
            "file": f"{g}.ncol"
        }
        _file_contents[_file].append(g)
    
    graphs_infos[contrast] = D

## Load data

In [7]:
_file_contents

defaultdict(list,
            {'network-C_D-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt': ['D-v-C.ALL.D',
              'D-v-C.ALL.C'],
             'network-C_H-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt': ['H-v-C.ALL.H',
              'H-v-C.ALL.C'],
             'network-C_HD-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt': ['HD-v-C.ALL.HD',
              'HD-v-C.ALL.C'],
             'network-H_HD-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt': ['HD-v-H.ALL.HD',
              'HD-v-H.ALL.H'],
             'network-C_W-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt': ['W-v-C.ALL.W',
              'W-v-C.ALL.C']})

In [8]:
known_cols = ['source', 'target', 'edge', 'group1', 'from', 'group2', 'to']
all_dfs = []
for contrast in design.contrasts:
    _file = design.files[contrast]

    treatments = contrast.split("-v-")
    
    df = pd.read_csv(input_dir / _file, sep="\t", comment="#", index_col=0)
    print(f"{_file}", df.shape)

    df['source'] = df["group1"].str[:2] +"."+ df["from"]
    df['target'] = df["group2"].str[:2] +"."+ df["to"]

    for g in _file_contents[_file]:
        col = graphs_infos[contrast]["treatments"][g]["treatment"]

        graph = df[['source', 'target', col]]
        graph.columns = ['source', 'target', g]
        graph.set_index(['source', 'target'], inplace=True)

        print("\t", col, g, "parsed")
        
        all_dfs.append(graph)

network-C_D-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt (2804, 8)
	 D D-v-C.ALL.D parsed
	 C D-v-C.ALL.C parsed
network-C_H-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt (2692, 8)
	 H H-v-C.ALL.H parsed
	 C H-v-C.ALL.C parsed
network-C_HD-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt (2972, 8)
	 HD HD-v-C.ALL.HD parsed
	 C HD-v-C.ALL.C parsed
network-H_HD-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt (2972, 8)
	 HD HD-v-H.ALL.HD parsed
	 H HD-v-H.ALL.H parsed
network-C_W-proteomics_metabolomics_hormonomics_qPCR_phenomics.txt (2804, 8)
	 W W-v-C.ALL.W parsed
	 C W-v-C.ALL.C parsed


## Functions for next steps

In [9]:
def count_nodes(df):
    nodes = set()
    nodes.update(df["source"])
    nodes.update(df["target"])
    return len(nodes)

def save_graph(df, prefix=''):
    
    folder =  graph_path
    folder.mkdir(exist_ok=True)

    print("graph\t\t # edges \t # pos edges\t # neg edges\tnode counts")
    for g in df: 
        w = df[g].copy().reset_index()
        w = w[w[g]!=0]
        w_pos = w[w[g]>0]
        w_neg = w[w[g]<0]

        w.to_csv(folder / f"{prefix}{g}.ncol", sep="\t", index=False, header=False)
        print(g, "\t", w.shape[0], "\t\t", w_pos.shape[0], "\t\t", w_neg.shape[0], end="\t\t")
        print(count_nodes(w), count_nodes(w_pos), count_nodes(w_neg))

def generate_threshold_stats(df, options="-l0 -u1 -i0.001 -m7,8", prefix=f""):
    t_stats_path = base_dir / "output" / "threshold-stats" 
    t_stats_path.mkdir(exist_ok=True)

    for g in df: 

        graph = graph_path / f"{prefix}{g}.ncol"
        result = t_stats_path / f"{g}"
        command = f"{THRESHOLD_EXE} {options} {graph} {result}"
        subprocess.run(command, shell=True)
            

In [10]:
all_df = pd.concat(all_dfs, axis=1)

In [11]:
save_graph(all_df, prefix='')#design.dataset+".")

graph		 # edges 	 # pos edges	 # neg edges	node counts
D-v-C.ALL.D 	 3028 		 1599 		 1205		90 86 86
D-v-C.ALL.C 	 3028 		 1405 		 1399		90 86 86
H-v-C.ALL.H 	 3028 		 1752 		 940		90 84 84
H-v-C.ALL.C 	 3028 		 1559 		 1133		90 84 84
HD-v-C.ALL.HD 	 3028 		 1651 		 1321		90 89 89
HD-v-C.ALL.C 	 3028 		 1490 		 1482		90 89 89
HD-v-H.ALL.HD 	 3028 		 1681 		 1291		90 89 89
HD-v-H.ALL.H 	 3028 		 1606 		 1366		90 89 89
W-v-C.ALL.W 	 3028 		 1503 		 1301		90 86 86
W-v-C.ALL.C 	 3028 		 1473 		 1331		90 86 86


# Differences


In [12]:
a = np.linspace([-1], [1], 11, axis=1); a
idx = [str(round(x, 2)) for x in a[0]]
show = pd.DataFrame(np.tile(a.T, (a.shape[1])) - np.tile(a, (a.shape[1], 1)), columns=idx, index=idx).T
show.index.name = "Control"
show.columns.name = "Treatment"
style = show.style.background_gradient(cmap='vlag', vmax=2, vmin=-2, axis=None)
style = style.format('{:.2f}')
style

Treatment,-1.0,-0.8,-0.6,-0.4,-0.2,0.0,0.2,0.4,0.6,0.8,1.0
Control,,,,,,,,,,,
-1.0,0.00,0.20,0.40,0.60,0.80,1.00,1.20,1.40,1.60,1.80,2.00
-0.8,-0.20,0.00,0.20,0.40,0.60,0.80,1.00,1.20,1.40,1.60,1.80
-0.6,-0.40,-0.20,0.00,0.20,0.40,0.60,0.80,1.00,1.20,1.40,1.60
-0.4,-0.60,-0.40,-0.20,0.00,0.20,0.40,0.60,0.80,1.00,1.20,1.40
-0.2,-0.80,-0.60,-0.40,-0.20,0.00,0.20,0.40,0.60,0.80,1.00,1.20
0.0,-1.00,-0.80,-0.60,-0.40,-0.20,0.00,0.20,0.40,0.60,0.80,1.00
0.2,-1.20,-1.00,-0.80,-0.60,-0.40,-0.20,0.00,0.20,0.40,0.60,0.80
0.4,-1.40,-1.20,-1.00,-0.80,-0.60,-0.40,-0.20,0.00,0.20,0.40,0.60
0.6,-1.60,-1.40,-1.20,-1.00,-0.80,-0.60,-0.40,-0.20,0.00,0.20,0.40


# pr-me-ho-qP-ph (as differences)

In [13]:
design.contrasts.items()

dict_items([('D-v-C', ('D-v-C.ALL.D', 'D-v-C.ALL.C')), ('H-v-C', ('H-v-C.ALL.H', 'H-v-C.ALL.C')), ('HD-v-C', ('HD-v-C.ALL.HD', 'HD-v-C.ALL.C')), ('HD-v-H', ('HD-v-H.ALL.HD', 'HD-v-H.ALL.H')), ('W-v-C', ('W-v-C.ALL.W', 'W-v-C.ALL.C'))])

In [14]:
all_df.fillna(0)

D-v-C.ALL.D  D-v-C.ALL.C  H-v-C.ALL.H  \
source   target                                                        
pr.PPD1  me.Glukose              -0.210707    -0.406558    -0.602601   
pr.PNSL2 me.Glukose               0.571539    -0.521793    -0.697149   
pr.LHCA3 me.Glukose               0.804591     0.751955    -0.534225   
pr.ndhI  me.Glukose              -0.680019    -0.684952    -0.393682   
pr.RPE   me.Glukose              -0.418123    -0.009752     0.000000   
...                                    ...          ...          ...   
pr.BSL1  ph.Fv.Fm                 0.000000     0.000000     0.000000   
         ph.qL                    0.000000     0.000000     0.000000   
         ph.deltaT                0.000000     0.000000     0.000000   
         ph.WATER.CONSUMPTION     0.000000     0.000000     0.000000   
         ph.FW_SH_Total           0.000000     0.000000     0.000000   

                               H-v-C.ALL.C  HD-v-C.ALL.HD  HD-v-C.ALL.C  \
source   target                                                           
pr.PPD1  me.Glukose               0.352423       0.396909     -0.406460   
pr.PNSL2 me.Glukose               0.179922      -0.726060     -0.527594   
pr.LHCA3 me.Glukose              -0.710419      -0.276094      0.746382   
pr.ndhI  me.Glukose               0.713748      -0.469467     -0.689760   
pr.RPE   me.Glukose               0.000000      -0.166716     -0.003861   
...                                    ...            ...           ...   
pr.BSL1  ph.Fv.Fm                 0.000000       0.000000      0.000000   
         ph.qL                    0.000000       0.000000      0.000000   
         ph.deltaT                0.000000       0.000000      0.000000   
         ph.WATER.CONSUMPTION     0.000000       0.000000      0.000000   
         ph.FW_SH_Total           0.000000       0.000000      0.000000   

                               HD-v-H.ALL.HD  HD-v-H.ALL.H  W-v-C.ALL.W  \
source   target                                                           
pr.PPD1  me.Glukose                 0.398961     -0.471930    -0.728503   
pr.PNSL2 me.Glukose                -0.726718     -0.726216     0.674853   
pr.LHCA3 me.Glukose                -0.276385     -0.264586    -0.733462   
pr.ndhI  me.Glukose                -0.470452     -0.504008     0.310769   
pr.RPE   me.Glukose                -0.171004     -0.944526     0.000000   
...                                      ...           ...          ...   
pr.BSL1  ph.Fv.Fm                  -0.753874      0.770052     0.299204   
         ph.qL                      0.433806      0.895104     0.291950   
         ph.deltaT                  0.198430     -0.685501    -0.177329   
         ph.WATER.CONSUMPTION      -0.670158      0.052870     0.242748   
         ph.FW_SH_Total             0.674165     -0.456680    -0.300029   

                               W-v-C.ALL.C  
source   target                             
pr.PPD1  me.Glukose              -0.497184  
pr.PNSL2 me.Glukose              -0.410174  
pr.LHCA3 me.Glukose              -0.767609  
pr.ndhI  me.Glukose               0.775729  
pr.RPE   me.Glukose               0.000000  
...                                    ...  
pr.BSL1  ph.Fv.Fm                 0.103984  
         ph.qL                    0.166139  
         ph.deltaT               -0.600671  
         ph.WATER.CONSUMPTION     0.089599  
         ph.FW_SH_Total          -0.344810  

[3028 rows x 10 columns]

In [15]:
all_diffs = []
for contrast, (condition, control) in design.contrasts.items():
    diff = all_df.fillna(0)[condition] - all_df.fillna(0)[control]
    diff.name = contrast
    all_diffs.append(diff)
    
    path = graph_path / f"{contrast}.ncol"
    diff.to_csv(path, sep="\t", header=False)
    print(path)

../output/original-graphs/D-v-C.ncol
../output/original-graphs/H-v-C.ncol
../output/original-graphs/HD-v-C.ncol
../output/original-graphs/HD-v-H.ncol
../output/original-graphs/W-v-C.ncol


In [16]:
all_diffs_df = pd.concat(all_diffs, axis=1)
all_diffs_df[all_diffs_df.isna().sum(axis=1)>0]

,,D-v-C,H-v-C,HD-v-C,HD-v-H,W-v-C
source,target,,,,,


In [17]:
all_diffs_df.sum()

D-v-C     359.420102
H-v-C     268.272681
HD-v-C    349.280299
HD-v-H     64.604375
W-v-C      61.993531
dtype: float64

In [19]:
save_graph(all_diffs_df)

graph		 # edges 	 # pos edges	 # neg edges	node counts
D-v-C 	 2804 		 1574 		 1230		86 86 86
H-v-C 	 2692 		 1569 		 1123		84 84 84
HD-v-C 	 2972 		 1667 		 1305		89 89 89
HD-v-H 	 2972 		 1522 		 1450		89 89 89
W-v-C 	 2804 		 1467 		 1337		86 86 86


In [22]:
generate_threshold_stats(all_diffs_df, options="-l0 -u2 -i0.001 -m7,8")


------------------------------------------------
input graph file:      ../output/original-graphs/D-v-C.ncol
output file prefix:    ../output/threshold-stats/D-v-C.15583.
lower threshold:       0
upper threshold:       2
threshold increment:   0.001
------------------------------------------------
Loading graph ... done.
Number vertices:  86
Number edges:     2804  (maximum possible number edges 3655)
------------------------------------------------

Iterative thresholding
Number steps: 2001

Step: 1, Threshold: 0		Vertices: 86	Edges: 2804
Step: 2, Threshold: 0.001		Vertices: 86	Edges: 2802
Step: 3, Threshold: 0.002		Vertices: 86	Edges: 2798
Step: 4, Threshold: 0.003		Vertices: 86	Edges: 2794
Step: 5, Threshold: 0.004		Vertices: 86	Edges: 2790
Step: 6, Threshold: 0.005		Vertices: 86	Edges: 2786
Step: 7, Threshold: 0.006		Vertices: 86	Edges: 2782
Step: 8, Threshold: 0.007		Vertices: 86	Edges: 2778
Step: 9, Threshold: 0.008		Vertices: 86	Edges: 2776
Step: 10, Threshold: 0.009		Vertices:

Step: 404, Threshold: 0.403		Vertices: 84	Edges: 1422
Step: 405, Threshold: 0.404		Vertices: 84	Edges: 1419
Step: 406, Threshold: 0.405		Vertices: 84	Edges: 1417
Step: 407, Threshold: 0.406		Vertices: 84	Edges: 1412
Step: 408, Threshold: 0.407		Vertices: 84	Edges: 1411
Step: 409, Threshold: 0.408		Vertices: 84	Edges: 1407
Step: 410, Threshold: 0.409		Vertices: 84	Edges: 1406
Step: 411, Threshold: 0.41		Vertices: 84	Edges: 1401
Step: 412, Threshold: 0.411		Vertices: 84	Edges: 1397
Step: 413, Threshold: 0.412		Vertices: 84	Edges: 1394
Step: 414, Threshold: 0.413 No edges removed, skipping. 
Step: 415, Threshold: 0.414		Vertices: 84	Edges: 1391
Step: 416, Threshold: 0.415		Vertices: 84	Edges: 1386
Step: 417, Threshold: 0.416		Vertices: 84	Edges: 1382
Step: 418, Threshold: 0.417		Vertices: 84	Edges: 1376
Step: 419, Threshold: 0.418		Vertices: 84	Edges: 1374
Step: 420, Threshold: 0.419		Vertices: 84	Edges: 1367
Step: 421, Threshold: 0.42		Vertices: 84	Edges: 1365
Step: 422, Threshold: 0.421

Step: 857, Threshold: 0.856		Vertices: 74	Edges: 389
Step: 858, Threshold: 0.857 No edges removed, skipping. 
Step: 859, Threshold: 0.858		Vertices: 74	Edges: 388
Step: 860, Threshold: 0.859		Vertices: 74	Edges: 385
Step: 861, Threshold: 0.86		Vertices: 74	Edges: 384
Step: 862, Threshold: 0.861		Vertices: 74	Edges: 382
Step: 863, Threshold: 0.862 No edges removed, skipping. 
Step: 864, Threshold: 0.863		Vertices: 74	Edges: 380
Step: 865, Threshold: 0.864		Vertices: 74	Edges: 379
Step: 866, Threshold: 0.865		Vertices: 74	Edges: 377
Step: 867, Threshold: 0.866 No edges removed, skipping. 
Step: 868, Threshold: 0.867		Vertices: 74	Edges: 375
Step: 869, Threshold: 0.868 No edges removed, skipping. 
Step: 870, Threshold: 0.869		Vertices: 74	Edges: 372
Step: 871, Threshold: 0.87		Vertices: 74	Edges: 371
Step: 872, Threshold: 0.871		Vertices: 74	Edges: 369
Step: 873, Threshold: 0.872		Vertices: 74	Edges: 367
Step: 874, Threshold: 0.873 No edges removed, skipping. 
Step: 875, Threshold: 0.874	

Step: 195, Threshold: 0.194		Vertices: 84	Edges: 1221
Step: 196, Threshold: 0.195		Vertices: 84	Edges: 1218
Step: 197, Threshold: 0.196		Vertices: 84	Edges: 1216
Step: 198, Threshold: 0.197 No edges removed, skipping. 
Step: 199, Threshold: 0.198 No edges removed, skipping. 
Step: 200, Threshold: 0.199		Vertices: 84	Edges: 1212
Step: 201, Threshold: 0.2		Vertices: 84	Edges: 1210
Step: 202, Threshold: 0.201		Vertices: 84	Edges: 1209
Step: 203, Threshold: 0.202		Vertices: 84	Edges: 1208
Step: 204, Threshold: 0.203		Vertices: 84	Edges: 1203
Step: 205, Threshold: 0.204		Vertices: 84	Edges: 1201
Step: 206, Threshold: 0.205		Vertices: 84	Edges: 1200
Step: 207, Threshold: 0.206		Vertices: 84	Edges: 1198
Step: 208, Threshold: 0.207 No edges removed, skipping. 
Step: 209, Threshold: 0.208		Vertices: 84	Edges: 1195
Step: 210, Threshold: 0.209		Vertices: 84	Edges: 1193
Step: 211, Threshold: 0.21		Vertices: 84	Edges: 1191
Step: 212, Threshold: 0.211		Vertices: 84	Edges: 1190
Step: 213, Threshold: 

Step: 838, Threshold: 0.837		Vertices: 70	Edges: 254
Step: 839, Threshold: 0.838 No edges removed, skipping. 
Step: 840, Threshold: 0.839 No edges removed, skipping. 
Step: 841, Threshold: 0.84		Vertices: 70	Edges: 252
Step: 842, Threshold: 0.841 No edges removed, skipping. 
Step: 843, Threshold: 0.842		Vertices: 70	Edges: 250
Step: 844, Threshold: 0.843		Vertices: 70	Edges: 249
Step: 845, Threshold: 0.844		Vertices: 70	Edges: 246
Step: 846, Threshold: 0.845		Vertices: 70	Edges: 244
Step: 847, Threshold: 0.846 No edges removed, skipping. 
Step: 848, Threshold: 0.847		Vertices: 70	Edges: 242
Step: 849, Threshold: 0.848 No edges removed, skipping. 
Step: 850, Threshold: 0.849		Vertices: 70	Edges: 240
Step: 851, Threshold: 0.85 No edges removed, skipping. 
Step: 852, Threshold: 0.851		Vertices: 70	Edges: 239
Step: 853, Threshold: 0.852		Vertices: 70	Edges: 235
Step: 854, Threshold: 0.853		Vertices: 70	Edges: 233
Step: 855, Threshold: 0.854		Vertices: 70	Edges: 232
Step: 856, Threshold: 0.

...

Step: 5, Threshold: 0.004		Vertices: 86	Edges: 1458
Step: 6, Threshold: 0.005		Vertices: 86	Edges: 1456
Step: 7, Threshold: 0.006		Vertices: 86	Edges: 1454
Step: 8, Threshold: 0.007		Vertices: 86	Edges: 1453
Step: 9, Threshold: 0.008		Vertices: 86	Edges: 1451
Step: 10, Threshold: 0.009		Vertices: 86	Edges: 1448
Step: 11, Threshold: 0.01		Vertices: 86	Edges: 1447
Step: 12, Threshold: 0.011		Vertices: 86	Edges: 1446
Step: 13, Threshold: 0.012 No edges removed, skipping. 
Step: 14, Threshold: 0.013		Vertices: 86	Edges: 1441
Step: 15, Threshold: 0.014		Vertices: 86	Edges: 1440
Step: 16, Threshold: 0.015		Vertices: 86	Edges: 1438
Step: 17, Threshold: 0.016		Vertices: 86	Edges: 1433
Step: 18, Threshold: 0.017		Vertices: 86	Edges: 1429
Step: 19, Threshold: 0.018		Vertices: 86	Edges: 1428
Step: 20, Threshold: 0.019		Vertices: 86	Edges: 1425
Step: 21, Threshold: 0.02		Vertices: 86	Edges: 1422
Step: 22, Threshold: 0.021		Vertices: 86	Edges: 1420
Step: 23, Threshold: 0.022		Vertices: 86	Edges: 14

Step: 220, Threshold: 0.219		Vertices: 86	Edges: 1060
Step: 221, Threshold: 0.22		Vertices: 86	Edges: 1058
Step: 222, Threshold: 0.221		Vertices: 86	Edges: 1056
Step: 223, Threshold: 0.222		Vertices: 86	Edges: 1052
Step: 224, Threshold: 0.223 No edges removed, skipping. 
Step: 225, Threshold: 0.224		Vertices: 86	Edges: 1050
Step: 226, Threshold: 0.225 No edges removed, skipping. 
Step: 227, Threshold: 0.226 No edges removed, skipping. 
Step: 228, Threshold: 0.227		Vertices: 86	Edges: 1049
Step: 229, Threshold: 0.228		Vertices: 86	Edges: 1048
Step: 230, Threshold: 0.229		Vertices: 86	Edges: 1046
Step: 231, Threshold: 0.23 No edges removed, skipping. 
Step: 232, Threshold: 0.231		Vertices: 86	Edges: 1045
Step: 233, Threshold: 0.232		Vertices: 86	Edges: 1044
Step: 234, Threshold: 0.233 No edges removed, skipping. 
Step: 235, Threshold: 0.234 No edges removed, skipping. 
Step: 236, Threshold: 0.235 No edges removed, skipping. 
Step: 237, Threshold: 0.236		Vertices: 86	Edges: 1042
Step: 238

Step: 939, Threshold: 0.938		Vertices: 81	Edges: 327
Step: 940, Threshold: 0.939		Vertices: 81	Edges: 326
Step: 941, Threshold: 0.94		Vertices: 81	Edges: 325
Step: 942, Threshold: 0.941		Vertices: 81	Edges: 323
Step: 943, Threshold: 0.942		Vertices: 81	Edges: 322
Step: 944, Threshold: 0.943 No edges removed, skipping. 
Step: 945, Threshold: 0.944 No edges removed, skipping. 
Step: 946, Threshold: 0.945		Vertices: 81	Edges: 321
Step: 947, Threshold: 0.946 No edges removed, skipping. 
Step: 948, Threshold: 0.947 No edges removed, skipping. 
Step: 949, Threshold: 0.948		Vertices: 81	Edges: 320
Step: 950, Threshold: 0.949 No edges removed, skipping. 
Step: 951, Threshold: 0.95		Vertices: 81	Edges: 318
Step: 952, Threshold: 0.951		Vertices: 81	Edges: 316
Step: 953, Threshold: 0.952 No edges removed, skipping. 
Step: 954, Threshold: 0.953		Vertices: 81	Edges: 315
Step: 955, Threshold: 0.954		Vertices: 81	Edges: 312
Step: 956, Threshold: 0.955		Vertices: 81	Edges: 311
Step: 957, Threshold: 0.

Step: 112, Threshold: 0.111 No edges removed, skipping. 
Step: 113, Threshold: 0.112		Vertices: 86	Edges: 1144
Step: 114, Threshold: 0.113 No edges removed, skipping. 
Step: 115, Threshold: 0.114		Vertices: 86	Edges: 1142
Step: 116, Threshold: 0.115		Vertices: 86	Edges: 1141
Step: 117, Threshold: 0.116		Vertices: 86	Edges: 1139
Step: 118, Threshold: 0.117		Vertices: 86	Edges: 1138
Step: 119, Threshold: 0.118		Vertices: 86	Edges: 1134
Step: 120, Threshold: 0.119		Vertices: 86	Edges: 1132
Step: 121, Threshold: 0.12 No edges removed, skipping. 
Step: 122, Threshold: 0.121		Vertices: 86	Edges: 1131
Step: 123, Threshold: 0.122		Vertices: 86	Edges: 1130
Step: 124, Threshold: 0.123		Vertices: 86	Edges: 1127
Step: 125, Threshold: 0.124 No edges removed, skipping. 
Step: 126, Threshold: 0.125		Vertices: 86	Edges: 1126
Step: 127, Threshold: 0.126 No edges removed, skipping. 
Step: 128, Threshold: 0.127		Vertices: 86	Edges: 1122
Step: 129, Threshold: 0.128		Vertices: 86	Edges: 1120
Step: 130, Thr

Step: 629, Threshold: 0.628 No edges removed, skipping. 
Step: 630, Threshold: 0.629 No edges removed, skipping. 
Step: 631, Threshold: 0.63		Vertices: 86	Edges: 529
Step: 632, Threshold: 0.631		Vertices: 86	Edges: 528
Step: 633, Threshold: 0.632 No edges removed, skipping. 
Step: 634, Threshold: 0.633		Vertices: 85	Edges: 527
Step: 635, Threshold: 0.634		Vertices: 85	Edges: 524
Step: 636, Threshold: 0.635		Vertices: 85	Edges: 523
Step: 637, Threshold: 0.636		Vertices: 85	Edges: 522
Step: 638, Threshold: 0.637 No edges removed, skipping. 
Step: 639, Threshold: 0.638 No edges removed, skipping. 
Step: 640, Threshold: 0.639		Vertices: 85	Edges: 521
Step: 641, Threshold: 0.64		Vertices: 85	Edges: 518
Step: 642, Threshold: 0.641 No edges removed, skipping. 
Step: 643, Threshold: 0.642		Vertices: 85	Edges: 517
Step: 644, Threshold: 0.643		Vertices: 85	Edges: 515
Step: 645, Threshold: 0.644		Vertices: 85	Edges: 512
Step: 646, Threshold: 0.645 No edges removed, skipping. 
Step: 647, Threshold

# END